In [1]:
import os
!export CUDA_VISIBLE_DEVICES=1
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List

import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
import torchvision.models as models

from torchprofile import profile_macs

assert torch.cuda.is_available(), \
"CUDA support is not available."

ImportError: Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.

In [2]:
"""
# Load pre-trained VGG16 model as the teacher model
teacher_model = VGG16(weights='imagenet', include_top=True)

# Create a smaller student model
def create_student_model(input_shape, num_classes):
    # Define your smaller student model architecture here
    # You can use fewer layers and/or smaller layer sizes compared to the teacher model
    # Make sure the architecture is capable of handling the same input_shape and num_classes

    return student_model

input_shape = (224, 224, 3)
num_classes = 1000

student_model = create_student_model(input_shape, num_classes)

# Define loss function and optimizer
loss_fn = CategoricalCrossentropy()
optimizer = Adam(learning_rate=1e-3)

# Compile the student model
student_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[CategoricalAccuracy()])

# Prepare your ImageNet data (e.g., using tf.keras.preprocessing.image.ImageDataGenerator)

# Perform distillation training
epochs = 10
batch_size = 32

for epoch in range(epochs):
    for batch_images, batch_labels in train_data_generator:
        with tf.GradientTape() as tape:
            # Get teacher predictions
            teacher_predictions = teacher_model(batch_images, training=False)
            
            # Get student predictions
            student_predictions = student_model(batch_images, training=True)
            
            # Calculate distillation loss (e.g., Mean Squared Error between teacher and student logits)
            distillation_loss = # Calculate distillation loss
            
            # Calculate classification loss for student model
            classification_loss = loss_fn(batch_labels, student_predictions)
            
            # Total loss is a combination of distillation and classification loss
            total_loss = distillation_loss + classification_loss
        
        # Calculate gradients and update student model
        gradients = tape.gradient(total_loss, student_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))
        
    print(f"Epoch {epoch+1}: Classification Loss = {classification_loss:.4f}, Distillation Loss = {distillation_loss:.4f}")

# Evaluate the student model
evaluation_results = student_model.evaluate(test_data_generator)
print("Evaluation results:", evaluation_results)
"""


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/d6/af/cb5ea6d1a9c83e715e29b45a598ebf542729cd216b43f5deefc27657bd38/tensorflow-2.13.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for keras<2.14,>=2.13.1 from https://files.pythonhosted.org/packages/2e/f3/19da7511b45e80216cbbd9467137b2d28919c58ba1ccb971435cb631e470/keras-2.13.1-py3-none-any.whl.metadata
  Using cached ker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 9.4 MB/s eta 0:00:00:00:0100:01
Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
Using cached libclang-16.0.6-py2.py3-none-manylinux2010_x86_64.whl (22.9 MB)
Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.4 MB/s eta 0:00:00:00:01
Using cached google_auth-2.22.0-py2.py3-none-any.whl (181 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 45.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.33.0
    Uninstalling google-auth-1.33.0:
      Successfully uninstalled google-auth-1.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [3]:
import tensorflow as tf

2023-08-31 20:47:40.410103: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 20:47:41.189297: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/shariff/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
print(tf.__version__)

2.13.0
